In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
from datetime import datetime, timedelta

def get_date_range(start_date, end_date):
    start_date = datetime.strptime(start_date, '%Y-%m-%d')
    end_date = datetime.strptime(end_date, '%Y-%m-%d')
    date_list = []
    
    current_date = start_date
    while current_date <= end_date:
        date_list.append(current_date.strftime('%Y-%m-%d'))
        current_date += timedelta(days=1)
    
    return date_list

def scrape_google_flights_all_prices(origin, destination, start_date, end_date, output_file):
    # Chrome 옵션 설정
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    )

    # 날짜 범위 생성
    date_list = get_date_range(start_date, end_date)
    
    # 전체 결과를 저장할 데이터프레임 초기화
    all_results = pd.DataFrame()

    for depart_date in date_list:
        print(f"Processing {depart_date}...")
        # Chrome WebDriver 실행
        driver = webdriver.Chrome(options=options)

        try:
            # Google Flights URL 생성
            url = f"https://www.google.com/travel/flights?q=Flights%20from%20{origin}%20to%20{destination}%20on%20{depart_date}"
            driver.get(url)
            
            # 페이지 로딩 대기
            time.sleep(60)

            # 전체 가격 정보 추출
            price_elements = driver.find_elements(By.XPATH, '//div[@jsname="IWWDBc"]//div[@class="U3gSDe"]//div[@class="BVAVmf I11szd POX3ye"]//div[contains(@class,"YMlIz FpEdX")]')
            prices = [price.text.replace("₩","").replace(",","") for price in price_elements]

            # 항공사 정보 추출
            airline_elements = driver.find_elements(By.XPATH, '//div[@jsname="IWWDBc"]//div[@class="OgQvJf nKlB3b"]//div[@class="KhL0De"]//div[@class="Ir0Voe"]//div[@class="sSHqwe tPgKwe ogfYpf"]')
            airlines = [airline.text.replace("\n"," ").replace("함께 예약된 다구간 항공권 ","") for airline in airline_elements]

            # 가격과 항공사 정보 필터링
            prices = list(filter(None, prices))
            airlines = list(filter(None, airlines))

            # 가격과 항공사 정보 매핑
            results = list(zip(airlines, prices))

            # 데이터프레임 생성
            if results:  # 결과가 있는 경우에만 처리
                df = pd.DataFrame(results, columns=["Airline", "Price"])
                df["Origin"] = origin
                df["Destination"] = destination
                df["Date"] = depart_date
                df = df[["Date", "Origin", "Destination", "Airline", "Price"]]
                
                # 결과를 전체 데이터프레임에 추가
                all_results = pd.concat([all_results, df], ignore_index=True)
                
                print(f"{depart_date} | {origin} → {destination} | 데이터 수집 완료")
            else:
                print(f"{depart_date} | 데이터가 없습니다")

        except Exception as e:
            print(f"{depart_date} | 데이터 수집 실패: {e}")
        finally:
            driver.quit()
            time.sleep(5)  # API 제한을 피하기 위한 대기 시간

    # 모든 결과를 하나의 CSV 파일로 저장
    if not all_results.empty:
        all_results.to_csv(output_file, index=False, encoding="utf-8")
        print(f"\n모든 데이터가 {output_file}에 저장되었습니다.")
    else:
        print("\n수집된 데이터가 없습니다.")

# 크롤링하는 날짜 정의하기
start = "2025-04-13"
end = "2026-03-31"

# 예시 실행
scrape_google_flights_all_prices(
    origin="ICN",
    destination="LAX",
    start_date=start,
    end_date=end,
    output_file=f"flights_all_ICN_LAX_{start}_{end}.csv"
)

Processing 2026-01-01...
2026-01-01 | ICN → LHR | 데이터 수집 완료
Processing 2026-01-02...
2026-01-02 | ICN → LHR | 데이터 수집 완료
Processing 2026-01-03...
2026-01-03 | ICN → LHR | 데이터 수집 완료
Processing 2026-01-04...
2026-01-04 | ICN → LHR | 데이터 수집 완료
Processing 2026-01-05...
2026-01-05 | ICN → LHR | 데이터 수집 완료
Processing 2026-01-06...
2026-01-06 | ICN → LHR | 데이터 수집 완료
Processing 2026-01-07...
2026-01-07 | ICN → LHR | 데이터 수집 완료
Processing 2026-01-08...
2026-01-08 | ICN → LHR | 데이터 수집 완료
Processing 2026-01-09...
2026-01-09 | ICN → LHR | 데이터 수집 완료
Processing 2026-01-10...
2026-01-10 | ICN → LHR | 데이터 수집 완료
Processing 2026-01-11...
2026-01-11 | ICN → LHR | 데이터 수집 완료
Processing 2026-01-12...
2026-01-12 | ICN → LHR | 데이터 수집 완료
Processing 2026-01-13...
2026-01-13 | ICN → LHR | 데이터 수집 완료
Processing 2026-01-14...
2026-01-14 | ICN → LHR | 데이터 수집 완료
Processing 2026-01-15...
2026-01-15 | ICN → LHR | 데이터 수집 완료
Processing 2026-01-16...
2026-01-16 | ICN → LHR | 데이터 수집 완료
Processing 2026-01-17...
2026-01-17 | IC